[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/029_pytorch_datasets/pytorch_datasets.ipynb)

# Pytorch - Datasets

En los posts anteriores hemos introducido los conceptos fundamentales de la librería de `Deep Learning` `Pytorch` y también hemos visto la funcionalidad que nos ofrece a la hora de diseñar y entrenar `redes neuronales`. En este post nos enfocamos en la herramientas que la librería nos da a la hora definir nuestros *datasets*.

In [28]:
import torch

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


## Iterando tensores

En los posts anteriores hemos utilizado el dataset MNIST para ilustrar los diferentes ejemplos que hemos visto. Vamos a seguir con este caso. A continuación tenemos una implementación en la que iteramos por los datos de manera explícita para entrenar nuestra red.

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
#from sklearn.datasets import fetch_openml
import numpy as np


mnist = pd.read_csv("/content/drive/MyDrive/Inteligencia Artificial 2/Datasets/Dig-MNIST.csv")

#X, Y = mnist["data"], mnist["target"]
#X.shape, Y.shape

# Obtener características (píxeles) y etiquetas (números)
X = mnist.iloc[:, 1:].values  # Seleccionar todas las columnas excepto la primera como características
Y = mnist.iloc[:, 0].values   # Seleccionar la primera columna como etiquetas

# Verificar las formas de X e Y
print("Forma de X:", X.shape)
print("Forma de Y:", Y.shape)

Forma de X: (10240, 784)
Forma de Y: (10240,)


In [31]:
# X_train, X_test, y_train, y_test = X[:60000] / 255., X[60000:] / 255., Y[:60000].astype(np.int), Y[60000:].astype(np.int)
x_2=np.array(X)
y_2=np.array(Y)

# normalización y split

X_train =x_2[:60000] / 255.
X_test =x_2[60000:] / 255.
y_train = y_2[:60000].astype(np.int32)
y_test = y_2[60000:].astype(np.int32)


X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy(y_train).long().cuda()

In [32]:
from sklearn.metrics import accuracy_score

def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

In [33]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 1000
log_each = 100
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 100/1000 Loss 0.75042
Epoch 200/1000 Loss 0.55166
Epoch 300/1000 Loss 0.44662
Epoch 400/1000 Loss 0.37837
Epoch 500/1000 Loss 0.32476
Epoch 600/1000 Loss 0.28353
Epoch 700/1000 Loss 0.25101
Epoch 800/1000 Loss 0.22483
Epoch 900/1000 Loss 0.20338
Epoch 1000/1000 Loss 0.18554


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


nan

## Iterando por *Batches*

En la implementación anterior estamos optimizando nuestro modelo con el algoritmo de `batch gradient descent`, en el que utilizamos todos nuestros datos en cada paso de optimización. Sin embargo, un algoritmo que puede converger más rápido (y única opción si nuestro dataset es tan grande que no cabe en memoria) es el de `mini-batch gradient descent` (el cual hemos ya utilizado en posts anteriores).

In [34]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 1000
batch_size = 10000
log_each = 100
l = []
model.train()
batches = len(X_t) // batch_size
for e in range(1, epochs+1):

    _l = []
    # iteramos por batches
    for b in range(batches):
        x_b = X_t[b*batch_size:(b+1)*batch_size]
        y_b = Y_t[b*batch_size:(b+1)*batch_size]

        # forward
        y_pred = model(x_b)

        # loss
        loss = criterion(y_pred, y_b)
        _l.append(loss.item())

        # ponemos a cero los gradientes
        optimizer.zero_grad()

        # Backprop (calculamos todos los gradientes automáticamente)
        loss.backward()

        # update de los pesos
        optimizer.step()

    l.append(np.mean(_l))
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 100/1000 Loss 0.73877
Epoch 200/1000 Loss 0.54211
Epoch 300/1000 Loss 0.43816
Epoch 400/1000 Loss 0.36661
Epoch 500/1000 Loss 0.31396
Epoch 600/1000 Loss 0.27355
Epoch 700/1000 Loss 0.24180
Epoch 800/1000 Loss 0.21633
Epoch 900/1000 Loss 0.19552
Epoch 1000/1000 Loss 0.17827


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


nan

Si bien esta implementación es correcta y funcional, dependiendo de nuestros datos puede llegar a complicarse mucho (por ejemplo, si necesitamos cargar muchas imágenes a las cuales queremos aplicar transformaciones, juntar en batches, etc...). Además, es común reutilizar la lógica para cargar nuestros datos no sólo para entrenar la red, si no para generar predicciones. Este hecho motiva el uso de las clases especiales que `Pytorch` nos ofrece para ello.

## La clase *Dataset*

La primera clase que tenemos que conocer es la clase `Dataset`. Esta clase hereda de la clase madre `torch.utils.data.Dataset` y tenemos que definir, como mínimo, tres funciones:

- `__init__`: el constructor
- `__len__`: devuelve el número de muestras en el dataset
- `__getitem__`: devuelve una muestra en concreto del dataset

Una vez definida la clase, ésta puede usarse como si de cualquier iterador se tratase.

In [35]:
# clase Dataset, hereda de la clase `torch.utils.data.Dataset`
# class DSPerros(torch.utils.data.Dataset):

class DatasetPersonalizado(torch.utils.data.Dataset):
    # constructor
    def __init__(self, X, Y):
        self.X = torch.from_numpy(X).float().cuda()
        self.Y = torch.from_numpy(Y).long().cuda()
    # devolvemos el número de datos en el dataset
    def __len__(self):
        return len(self.X)
        # return len(self.Y)
    # devolvemos el elemento `ix` del dataset
    def __getitem__(self, ix):
        return self.X[ix], self.Y[ix]

Una vez definida la clase, podemos instanciar un objeto que podemos usar para iterar por nuestros datos.

In [36]:
dataset = DatasetPersonalizado(X_train, y_train)

len(dataset)

10240

In [37]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 1000
batch_size = 10000
log_each = 1
l = []
model.train()
batches = len(dataset) // batch_size
for e in range(1, epochs+1):

    _l = []
    # iteramos por batches en el dataset
    for b in range(batches):
        x_b, y_b = dataset[b*batch_size:(b+1)*batch_size]

        # forward
        y_pred = model(x_b)

        # loss
        loss = criterion(y_pred, y_b)
        _l.append(loss.item())

        # ponemos a cero los gradientes
        optimizer.zero_grad()

        # Backprop (calculamos todos los gradientes automáticamente)
        loss.backward()

        # update de los pesos
        optimizer.step()

    l.append(np.mean(_l))
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 1/1000 Loss 2.30002
Epoch 2/1000 Loss 2.25754
Epoch 3/1000 Loss 2.21018
Epoch 4/1000 Loss 2.15285
Epoch 5/1000 Loss 2.08538
Epoch 6/1000 Loss 2.01026
Epoch 7/1000 Loss 1.93124
Epoch 8/1000 Loss 1.85196
Epoch 9/1000 Loss 1.77537
Epoch 10/1000 Loss 1.70345
Epoch 11/1000 Loss 1.63729
Epoch 12/1000 Loss 1.57772
Epoch 13/1000 Loss 1.52914
Epoch 14/1000 Loss 1.51564
Epoch 15/1000 Loss 1.53858
Epoch 16/1000 Loss 1.54618
Epoch 17/1000 Loss 1.52913
Epoch 18/1000 Loss 1.50026
Epoch 19/1000 Loss 1.46818
Epoch 20/1000 Loss 1.43743
Epoch 21/1000 Loss 1.40965
Epoch 22/1000 Loss 1.38614
Epoch 23/1000 Loss 1.36377
Epoch 24/1000 Loss 1.34291
Epoch 25/1000 Loss 1.32029
Epoch 26/1000 Loss 1.29807
Epoch 27/1000 Loss 1.27613
Epoch 28/1000 Loss 1.25523
Epoch 29/1000 Loss 1.23531
Epoch 30/1000 Loss 1.21641
Epoch 31/1000 Loss 1.19868
Epoch 32/1000 Loss 1.18173
Epoch 33/1000 Loss 1.16601
Epoch 34/1000 Loss 1.15078
Epoch 35/1000 Loss 1.13670
Epoch 36/1000 Loss 1.12294
Epoch 37/1000 Loss 1.11009
Epoch 38/1

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


nan

Podemos iterar directamente sobre el objeto `dataset` de la misma manera que hacíamos anteriormente, sin embargo `Pytorch` no ofrece otro objeto que nos facilita las cosas a la hora de iterar por batches.

## La clase *DataLoader*

La clase `DataLoader` recibe un `Dataset` e implementa la lógica para iterar nuestros datos en batches.

In [38]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=62, shuffle=True)

In [39]:
x, y = next(iter(dataloader))

x.shape, y.shape
# x[0], y[0]

# x, y = next(iter(dataloader))
# x.shape, y.shape
# x[0], y[0]

(torch.Size([62, 784]), torch.Size([62]))

También permite mezclar los datos al principio de cada epoch con el parámetro `shuffle`, de manera automática carga nuestros datos de manera optimizada utilizando varios *cores* de nuestra CPU si es posible, etc.

In [40]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 1000
log_each = 100
l = []
model.train()
for e in range(1, epochs+1):

    _l = []
    # iteramos por batches en el dataloader
    for x_b, y_b in dataloader:

        # forward
        y_pred = model(x_b)

        # loss
        loss = criterion(y_pred, y_b)
        _l.append(loss.item())

        # ponemos a cero los gradientes
        optimizer.zero_grad()

        # Backprop (calculamos todos los gradientes automáticamente)
        loss.backward()

        # update de los pesos
        optimizer.step()

    l.append(np.mean(_l))
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 100/1000 Loss 0.02113
Epoch 200/1000 Loss 0.01066
Epoch 300/1000 Loss 0.00714
Epoch 400/1000 Loss 0.00538
Epoch 500/1000 Loss 0.00431
Epoch 600/1000 Loss 0.00360
Epoch 700/1000 Loss 0.00309
Epoch 800/1000 Loss 0.00271
Epoch 900/1000 Loss 0.00241
Epoch 1000/1000 Loss 0.00217


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


nan

También permite definir nuestra propia lógica para crear los batches, algo que puede ser útil en ciertas ocasiones.

In [41]:
def collate_fn(batch):
    return torch.stack([x for x, y in batch]), torch.stack([y for x, y in batch]), torch.stack([2.*x for x, y in batch])

In [42]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True, collate_fn=collate_fn)

In [44]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 1000
log_each = 100
l = []
model.train()
for e in range(1, epochs+1):

    _l = []
    # iteramos por batches en el dataloader
    # no usamos x2_b, sólo es para ver un ejemplo
    for x_b, y_b, x2_b in dataloader:

        # forward
          y_pred = model(x_b)

        # loss
          loss = criterion(y_pred, y_b)
          _l.append(loss.item())

        # ponemos a cero los gradientes
          optimizer.zero_grad()

        # Backprop (calculamos todos los gradientes automáticamente)
          loss.backward()

        # update de los pesos
          optimizer.step()

    l.append(np.mean(_l))
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 100/1000 Loss 0.02718
Epoch 200/1000 Loss 0.01376
Epoch 300/1000 Loss 0.00923
Epoch 400/1000 Loss 0.00696
Epoch 500/1000 Loss 0.00558
Epoch 600/1000 Loss 0.00466
Epoch 700/1000 Loss 0.00401
Epoch 800/1000 Loss 0.00351
Epoch 900/1000 Loss 0.00313
Epoch 1000/1000 Loss 0.00282


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


nan

## Resumen

En este post hemos visto diferentes maneras en las que podemos iterar por nuestros datos para entrenar un modelo en `Pytorch`. Si nuestro dataset es sencillo y podemos representarlo como un simple `array` de `NumPy` podemos iterar directamente el `array`, transformándolo previamente en un `tensor`. Sin embargo, cuando nuestro dataset sea más grande y no quepa en memoria o necesite cierto pre-proceso o transformaciones, es muy conveniente utilizar las clases que `Pytorch` nos ofrece para ello. Estas clases son, principalmente, el `Dataset` y el `DataLoader`, las cuales nos van a permitir iterar por nuestros datos de manera eficiente y generar *batches* de forma sencilla (además de otras funcionalidades como mezclar los datos al principio de cada epoch, cargar datos en paralelo, etc).